In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from langchain.chat_models import init_chat_model

model = init_chat_model(
    "gemini-2.5-flash",
    model_provider="google_genai",
    temperature=0,
)
response = model.invoke("Hola como estas?")
response.text


E0000 00:00:1761653230.441525    4954 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


'Hola, estoy muy bien, gracias. ¿Y tú, cómo estás?'

In [5]:
response = model.invoke("Que clima hace en la ciudad de Bogota? ")
response.pretty_print()

================================== Ai Message ==================================

Bogotá tiene un clima **frío de alta montaña**, caracterizado por ser fresco a frío durante todo el año, con temperaturas que no varían mucho entre estaciones.

Aquí te detallo sus características principales:

1.  **Temperaturas:**
    *   Las temperaturas promedio oscilan entre los **8°C y 19°C**.
    *   Durante el día, suele ser fresco, con promedios de 14-18°C.
    *   Por la noche, las temperaturas bajan considerablemente, pudiendo llegar a los 5-10°C.
    *   Rara vez hace mucho calor o mucho frío extremo (bajo cero).

2.  **Lluvias:**
    *   Es una ciudad con **frecuentes lluvias**, que pueden ser lloviznas persistentes o aguaceros fuertes.
    *   Tiene dos temporadas de lluvias principales:
        *   De **marzo a mayo**.
        *   De **septiembre a noviembre**.
    *   Las temporadas "secas" (diciembre-febrero y junio-agosto) suelen tener menos lluvia, pero aún así pueden presentarse precip

In [7]:
system_prompt = """
Eres un asistente de ventas que ayuda a los clientes a encontrar los productos que necesitan.

Y tus productos son:

- Computadora Gamer
- Mouse Gamer
- Teclado Gamer
- Auriculares Gamer
- Mousepad Gamer
- Silla Gamer
"""
messages = [
    ("system", system_prompt),
    ("user", "Dime los productos que ofreces en la tienda?")
]
response = model.invoke(messages)
response.pretty_print()


================================== Ai Message ==================================

¡Claro! En nuestra tienda, ofrecemos una amplia gama de productos para mejorar tu experiencia de juego. Aquí tienes lo que tenemos disponible:

*   **Computadora Gamer**
*   **Mouse Gamer**
*   **Teclado Gamer**
*   **Auriculares Gamer**
*   **Mousepad Gamer**
*   **Silla Gamer**

¿Hay algún producto en particular que te interese o te gustaría saber más sobre alguno de ellos?


In [24]:
from langchain.tools import tool
import requests

@tool("get_products", description="Devuelve los productos que ofrece la tienda y su precio")
def get_products():
    """Devuelve los productos que ofrece la tienda"""
    response = requests.get(f"https://api.escuelajs.co/api/v1/products")
    products = response.json()
    return "\n".join([f"{product['title']} - {product['price']}" for product in products])


In [25]:
get_products.invoke({"price": 10})

'Futuristic Holographic Soccer Cleats - 39\nRainbow Glitter High Heels - 39\nChic Summer Denim Espadrille Sandals - 33\nVibrant Pink Classic Sneakers - 84\nFuturistic Chic High-Heel Boots - 36\nElegant Purple Leather Loafers - 17\nClassic Blue Suede Casual Shoes - 39\nSleek Futuristic Electric Bicycle - 22\nRadiant Citrus Eau de Parfum - 73\nSleek Olive Green Hardshell Carry-On Luggage - 48\nTrendy Pink-Tinted Sunglasses - 38\nstring - 100\nGulab jamun 1 kg - 300\ntes - 22\nParfum - 22\nParfum Cologne - 18\nAPITestProduct - 123\nTest Product 123 - 123\nTest Product Auto - 123\nAutomation Test Product - 123\nSample Product - 20\nSample Productsasa - 20\nSample Productsasaw - 20\nTest Product - 555\nSample Productsasawsds - 20\nSample Productsasawsdss - 20\nTestProd - 111\nuser_1761637503730 - 10\nuser_1761637716208 - 10\nuser_1761637863905 - 10\nuser_1761637930562 - 10\nuser_1761638348265 - 10\nuser_1761638544729 - 10\nuser_1761638738143 - 10\nuser_1761638937202 - 10\nuser_1761639028345

In [27]:
@tool("get_weather", description="Devuelve el clima de la ciudad")
def get_weather(city: str):
    """Devuelve el clima de la ciudad"""
    response = requests.get(f"https://geocoding-api.open-meteo.com/v1/search?name={city}&count=10&language=es&format=json")
    data = response.json()
    latitude = data["results"][0]["latitude"]
    longitude = data["results"][0]["longitude"]
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&daily=weathercode,temperature_2m_max,temperature_2m_min&timezone=Europe%2FBerlin")
    data = response.json()
    response = f"El clima en {city} es {data['daily']['weathercode'][0]} y la temperatura es de {data['daily']['temperature_2m_max'][0]}C a {data['daily']['temperature_2m_min'][0]}C"
    return response

get_weather.invoke({"city": "Bogota"})

'El clima en Bogota es 3 y la temperatura es de 19.4C a 12.6C'

In [29]:
system_prompt = """
Eres un asistente de ventas que ayuda a los clientes a encontrar los productos que necesitan y dar el clima de la ciudad.

Tus tools son:

- get_products: Devuelve los productos que ofrece la tienda y su precio
- get_weather: Devuelve el clima de la ciudad
"""
messages = [
    ("system", system_prompt),
    ("user", "Dime los productos que ofreces en la tienda?")
]
model_with_tools = model.bind_tools([get_products, get_weather])
response = model_with_tools.invoke(messages)
response.tool_calls


[{'name': 'get_products',
  'args': {},
  'id': 'e66e31e4-7b02-4bbb-bdc9-cf9787ed6621',
  'type': 'tool_call'}]

In [30]:
messages = [
    ("system", system_prompt),
    ("user", "Hola que tal?")
]
response = model_with_tools.invoke(messages)
response.pretty_print()

================================== Ai Message ==================================

[{'type': 'text', 'text': '¡Hola! Estoy aquí para ayudarte. ¿En qué puedo asistirte hoy?', 'extras': {'signature': 'Ct0BAdHtim8pUMgPBQNHEJyiN07IcZMdojT80EM7tacrpwg0mfdPOViUKCUTxTtQshV60xtCjyYmrzMA3sHVNPGh367HbbuG7qw0DZ1AZndIfWCZVrgNO4/xpcL4WpeODDp/ttw1Z+FErlilOQl29xgF91ef/gXUEmEj++qlbYIo1ALPmtH15DIh4QmI2TbwUhJqwYQbguARObR4pz1zE70RUKe5ZuN3TBVIy9pqkL5V9I7J3YznocSypyUqnqkMMbGx5QUhSkO/F7C2OnHAu1EzK6rSbFdLw+kppoxP4iQ='}}]


In [32]:
system_prompt = """
Eres un asistente de ventas que ayuda a los clientes a encontrar los productos que necesitan y dar el clima de la ciudad.

Tus tools son:

- get_products: Devuelve los productos que ofrece la tienda y su precio
- get_weather: Devuelve el clima de la ciudad
"""
messages = [
    ("system", system_prompt),
    ("user", "Cual es el clima en la capital de Colombia?")
]
model_with_tools = model.bind_tools([get_products, get_weather])
response = model_with_tools.invoke(messages)
response.tool_calls


[{'name': 'get_weather',
  'args': {'city': 'Bogotá'},
  'id': '2796a2c9-c1e2-47f9-b6b3-8af60f60559c',
  'type': 'tool_call'}]